In [2]:
import bempp.api
from bempp.api.assembly.blocked_operator import BlockedOperator
import math
import numpy as np
import scipy 
import cmath
from numba import objmode
from numpy.linalg import slogdet
from bempp.api.operators.far_field import helmholtz as helmholtz_farfield
import matplotlib.pyplot as plt

In [3]:
N_wn = 20
lbound_wn = 0
ubound_wn = 10
quad_points = np.linspace(np.exp(-ubound_wn), np.exp(-lbound_wn), N_wn)

In [4]:
h_coarse = 0.1
h_fine = 0.05

In [5]:
Cas_exact = []
Cas_05_005 = []
Cas_05_01 = []

R = h_coarse/h_fine

R1 = 0.5
R2 = 1
for dist in [0.5, 0.75, 1, 1.25, 1.5, 1.75]:
    CasE_05 = []
    for h in [h_coarse, h_fine]:

        results = []

        sphere1 = bempp.api.shapes.sphere(r = R1, h=h, origin=((dist/2) + 0.5, 0, 0))
        sphere2 = bempp.api.shapes.sphere(r = R2, h=h, origin=(-((dist/2) + 1), 0, 0))

        space_sphere1 = bempp.api.function_space(sphere1, "P", 1)
        space_sphere2 = bempp.api.function_space(sphere2, "P", 1)

        for index2, point2 in enumerate(quad_points):

            wavenumber = -1j * np.log(point2)

            slp11 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere1, space_sphere1, wavenumber)
            slp12 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere1, space_sphere1, wavenumber)
            slp21 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere2 , space_sphere2 , wavenumber)
            slp22 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere2 , space_sphere2 , wavenumber)

            mat11 = slp11.weak_form().A
            mat12 = slp12.weak_form().A
            mat21 = slp21.weak_form().A
            mat22 = slp22.weak_form().A
            mat12_zero = np.zeros((mat11.shape[0],mat12.shape[1]))
            mat21_zero = np.zeros((mat22.shape[0],mat11.shape[1]))

            mat = np.block([[mat11,mat12],[mat21,mat22]])  
            mat11_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat11), np.eye(mat11.shape[0]))
            mat22_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat22), np.eye(mat22.shape[0]))

            inv_free_mat2 = np.block([[mat11_inv,  mat12_zero],
                                  [mat21_zero, mat22_inv]])  

            combined_inverse_free = inv_free_mat2.dot(mat)
            sign, logdet = slogdet(combined_inverse_free)

            results.append(logdet)

        CasE_05.append(np.trapz(-np.array(results)/quad_points, quad_points)/(2*np.pi))
        
    Cas_exact.append(CasE_05[1] + (CasE_05[1] - CasE_05[0])/(R**2 - 1))
    Cas_05_005.append(CasE_05[1])
    Cas_05_01.append(CasE_05[0])
    print(dist, CasE_05[1] + (CasE_05[1] - CasE_05[0])/(R**2 - 1), CasE_05[1], CasE_05[0])

0.5 0.029989228541067104 0.029860930652490823 0.02947603698676198
0.75 0.01314608221096681 0.013105308164410665 0.012982986024742233
1 0.007251675311940183 0.007232735178537544 0.007175914778329626
1.25 0.004519072233112009 0.004508664783327096 0.004477442433972357
1.5 0.003046268011981152 0.0030398167362710037 0.003020462909140558
1.75 0.0021691170225390756 0.0021648375935628106 0.0021519993066340163


In [27]:
R1 = 0.5
R2 = 1
center_dist = np.linspace(R1 + R2 + 0.5, R1 + R2 + 3, 11)
eta_ = R1/R2

coeff_list  = [-eta_/4, -(eta_ + eta_**2)/8, -(34*(eta_ + eta_**3) + 9*(eta_**2))/48,
              -(2*(eta_ + eta_**4) + 23*(eta_**2 + eta_**3))/32,
              -(8352*(eta_ + eta_**5) + 1995*(eta_**2 + eta_**4) + 38980*eta_**3)/5760,
              -(-1344*(eta_ + eta_**6) + 5478*(eta_**2 + eta_**5)+ 2357*(eta_**3 + eta_**4))/2304]

In [28]:
asy_casimir_energy_sphere_sphere = np.empty(11, dtype='float64')

for i in range(11):
    asy_casimir_energy = 0
    for j in range(len(coeff_list)):
        asy_casimir_energy += (1/center_dist[i]) * coeff_list[j] * (R2/center_dist[i]**(j+2))
    asy_casimir_energy_sphere_sphere[i] = -asy_casimir_energy/np.pi

In [29]:
asy_casimir_energy_sphere_sphere

array([0.01820843, 0.01025902, 0.00631957, 0.00416607, 0.00289457,
       0.00209628, 0.00156958, 0.00120763, 0.00095035, 0.00076222,
       0.00062129])

In [ ]:
asy_casimir_energy = 0
for j in range(len(coeff_list)):
    asy_casimir_energy += (1/l) * coeff_list[j] * (R1/l**(j+2))
print(-asy_casimir_energy/np.pi)

In [ ]:
h = 0.1 dist = 0.5 0.05931734269526816 asy 0.0182115933588116
h = 0.1 dist = 1.5 0.006584081248087261 asy 0.005129512234766054
h = 0.1 dist= 3.0 0.0015330308840655895 asy 0.0014472844993857507

In [ ]:
0.04372859968911746 

In [ ]:
CasE_05 = [0.044142920889365764, 0.04378745428754526, 0.043307825800813826, 0.0425274735585427] 
CasE_15 = [0.004684510596741076, 0.004668668075048978, 0.0046463126764854845, 0.004608327156406329] 
CasE_30 = [0.001029159937793544, 0.0010266636526601919, 0.0010231501929812861, 0.0010171903056843653]

In [ ]:
Rel_err_05 = [abs(CasE_05[0] - CasE_05[1])/CasE_05[0], abs(CasE_05[0] - CasE_05[2])/CasE_05[0], abs(CasE_05[0] - CasE_05[3])/CasE_05[0]]
Rel_err_15 = [abs(CasE_15[0] - CasE_15[1])/CasE_15[0], abs(CasE_15[0] - CasE_15[2])/CasE_15[0], abs(CasE_15[0] - CasE_15[3])/CasE_15[0]]
Rel_err_30 = [abs(CasE_30[0] - CasE_30[1])/CasE_30[0], abs(CasE_30[0] - CasE_30[2])/CasE_30[0], abs(CasE_30[0] - CasE_30[3])/CasE_30[0]]

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(21, 6))
ax1.plot([0.1, 0.15, 0.2], Rel_err_05,'*-')
ax1.set_xlabel('Grid size $h$', fontsize = 20)
ax1.set_ylabel('Relative error', fontsize = 15)
ax1.set_title('dist = 0.5', fontsize = 20)

ax2.plot([0.1, 0.15, 0.2], Rel_err_15,'*-')
ax2.set_xlabel('Grid size $h$', fontsize = 20)
ax2.set_ylabel('Relative error', fontsize = 15)
ax2.set_title('dist = 1.5', fontsize = 20)

ax3.plot([0.1, 0.15, 0.2], Rel_err_15,'*-')
ax3.set_xlabel('Grid size $h$', fontsize = 20)
ax3.set_ylabel('Relative error', fontsize = 15)
ax3.set_title('dist = 3.0', fontsize = 20)


fig.tight_layout()
fig.suptitle('Relative error in the scalar case', fontsize = 25)
fig.subplots_adjust(top=0.85)
fig.savefig('CasE_scalar.pdf')